# numba-rvsdg demo

This is an introduction to the use of the `numba_rvsdg` package. Here, we show how to convert Python source as Abstract Syntax Tree (AST) into a Control Flow Graph (CFG) and then into a Structured Control Flow Graph and finally back into Python source as Abstract Syntax Tree. Specifically, this demos the use of the AST -> SCFG -> AST transformer pipeline.

The code in this demo is a faithful implementation of the Algorithms described in `Bahmann2015` -- https://dl.acm.org/doi/pdf/10.1145/2693261

## Motiviation

* Abstract away control flow
* Make it easier for compilers to optimize
* Beginning of a source frontend for Numba

## Overview

The transformer pipeline consists of the following steps:

1. Convert Python AST into a plain CFG. (This also prunes unreachable, empty and no-op nodes (`pass` and `...` (ellipsis))).
2. Apply the following transforms to go from CFG to SCFG:
    1. `CLOSE_CFG` :: this will add edges and a node such that the CFG only has a single exiting node. The graph is then considered _closed_.
    2. `LOOP_RESTRUCTURE` :: This will perform a type of loop rotation such that every loop has a single header, a single exiting latch and a single backedge to the header. The loop is then considered _closed_.
    3. `BRANCH_RESTRUCTURE` :: This will partition the graph such that every branching flow control construct is divided into three types of `regions`. `HEAD` regions that are a linear set of instructions that conclude with a branch/jump. Two or more `BRANCH` regions that contain the code for each control flow path taken. And finally a single `TAIL` region that all `BRANCH` regions fall through to.
3. Lastly we synthesize an equivalent Python program which is structurally different but behaviourally equivalent. That is to say the same input will yield the same output, (barring any stochastic programs). This program can then be run by the interpreter.

Note that some AST level rewrites were employed in order to desugar the Python `for` loop semantics into the CFG style formalism of basic blocks and edges between them.

Note also that the flow control of Python `Exceptions` is not yet handled by this pipeline (as of 2024-06-14).

## Preliminaries

In [1]:
import ast

import IPython

from numba_rvsdg.core.datastructures.ast_transforms import AST2SCFGTransformer, SCFG2ASTTransformer, unparse_code
from numba_rvsdg.rendering.rendering import SCFGRenderer

def render_scfg_info_notebook(scfg):
    """Render graphviz `dot` output into notebook."""
    IPython.display.display_svg(SCFGRenderer(scfg).g)

def render_scfg_info_notebook_center(scfg):
    svg_data = SCFGRenderer(scfg).g._repr_image_svg_xml()
    html = f"""
    <div style="display: flex; justify-content: center;">
        {svg_data}
    </div>
    """
    IPython.display.display(IPython.display.HTML(html))

# https://github.com/ipython/ipython/issues/11747#issuecomment-528694702
def display_source(code):
    def _jupyterlab_repr_html_(self):
        from pygments import highlight
        from pygments.formatters import HtmlFormatter

        fmt = HtmlFormatter()
        style = "<style>{}\n{}</style>".format(
            fmt.get_style_defs(".output_html"), fmt.get_style_defs(".jp-RenderedHTML")
        )
        return style + highlight(self.data, self._get_lexer(), fmt)

    # Replace _repr_html_ with our own version that adds the 'jp-RenderedHTML' class
    # in addition to 'output_html'.
    IPython.display.Code._repr_html_ = _jupyterlab_repr_html_
    return IPython.display.Code(data=code, language="python3")

def compile_ast(ast_):
    """Custom `compile` function via `exec`. """
    exec_locals = {}
    name = ast_.name
    exec(ast.unparse(ast_), {}, exec_locals)
    transformed = exec_locals[name]
    return transformed

# https://github.com/ipython/ipython/issues/11747#issuecomment-528694702
def display_source(code):
    def _jupyterlab_repr_html_(self):
        from pygments import highlight
        from pygments.formatters import HtmlFormatter

        fmt = HtmlFormatter()
        style = "<style>{}\n{}</style>".format(
            fmt.get_style_defs(".output_html"), fmt.get_style_defs(".jp-RenderedHTML")
        )
        return style + highlight(self.data, self._get_lexer(), fmt)

    # Replace _repr_html_ with our own version that adds the 'jp-RenderedHTML' class
    # in addition to 'output_html'.
    IPython.display.Code._repr_html_ = _jupyterlab_repr_html_
    return IPython.display.Code(data=code, language="python3")

## Branch Restructure: Example Function and CFG
Let's begin with a simple branching function. This will show the basics of the `BRANCH_RESTRUCTURE`.

In [2]:
def branch(b: int) -> int:
    if b:
        r = 1
    else:
        r = 2
    return r

In [3]:
scfg = AST2SCFGTransformer(branch).transform_to_SCFG()

In [4]:
render_scfg_info_notebook_center(scfg)

## Branch Restructure: Running the Restructuring

In [5]:
scfg.restructure()

In [6]:
render_scfg_info_notebook_center(scfg)

## Branch Restructure: Python Synthesis

Now, to complete the pipeline, let's synthesize some Python code from this structure.

In [7]:
original_ast = unparse_code(branch)[0]
transformed_ast  = SCFG2ASTTransformer().transform(original=original_ast, scfg=scfg)
display_source(ast.unparse(transformed_ast))

def transformed_branch(b: int) -> int:
    if b:
        r = 1
    else:
        r = 2
    return r

In [8]:
transformed_branch = compile_ast(transformed_ast)
transformed_branch(0), transformed_branch(1)

(2, 1)

## Close CFG: Example and CFG

Noted, the above example is a bit silly, since we haven't actually altered the initial program. So let's try something a bit more involved.

In [9]:
def multi_return(b: int) -> int:
    if b:
        return 1
    else:
        return 2

In [10]:
scfg = AST2SCFGTransformer(multi_return).transform_to_SCFG()

In [11]:
render_scfg_info_notebook_center(scfg)

We can see from the graph, that it is not closed -- there are two nodes without outgoing edges. Let's restructure.

## Close CFG: Restructure

In [12]:
scfg.restructure()

In [13]:
render_scfg_info_notebook_center(scfg)

As you can see, an additional node was generated and the CFG is now closed. Let's synthesize some Python and see how this looks.

## Close CFG: Python Synthesis

In [14]:
original_ast = unparse_code(multi_return)[0]
transformed_ast  = SCFG2ASTTransformer().transform(original=original_ast, scfg=scfg)
display_source(ast.unparse(transformed_ast))

def transformed_multi_return(b: int) -> int:
    if b:
        __scfg_return_value__ = 1
    else:
        __scfg_return_value__ = 2
    return __scfg_return_value__

In [15]:
transformed_multi_return = compile_ast(transformed_ast)
transformed_multi_return(0), transformed_multi_return(1)

(2, 1)

As you can see, the conversion was fine and we now have a Python program with only a single `return` statement.

## Loop Restructure: Example and CFG

Now, let's have a look at a simple while loop:

In [16]:
def while_loop():
    c = 0
    while c < 10:
        c += 3
    return c

In [17]:
scfg = AST2SCFGTransformer(while_loop).transform_to_SCFG()

In [18]:
render_scfg_info_notebook_center(scfg)

## Loop Restructure: Restructure

In [19]:
scfg.restructure()

In [20]:
render_scfg_info_notebook_center(scfg)

As you can see, the loop was placed into a region, the loop is closed and it is tail-controlled. That is to say there is a single backedge from the exiting latch. 

## Loop Restructure: Python Synthesis

In [21]:
original_ast = unparse_code(while_loop)[0]
transformed_ast  = SCFG2ASTTransformer().transform(original=original_ast, scfg=scfg)
display_source(ast.unparse(transformed_ast))

def transformed_while_loop():
    c = 0
    __scfg_loop_cont__ = True
    while __scfg_loop_cont__:
        if c < 10:
            c += 3
            __scfg_backedge_var_0__ = 0
        else:
            __scfg_backedge_var_0__ = 1
        __scfg_loop_cont__ = not __scfg_backedge_var_0__
    return c

As you can see, the loop is now conditioned on a variable `__scfg_backedge_var_0__`. It's not exactly a _tail controlled_ loop since Python doesn't support a _do-while_ construct but this is a good approximation of it.

## Early Exit: Example and CFG

Things become more interesting, when the loop contains early exits:

In [22]:
def while_loop_with_exit(a: int):
    c = 0
    while c < 10:
        c += 3
        if c > a:
            return c + 1
    return c

In [23]:
scfg = AST2SCFGTransformer(while_loop_with_exit).transform_to_SCFG()

In [24]:
render_scfg_info_notebook_center(scfg)

## Early Exit: Restructure

In [25]:
scfg.restructure()

In [26]:
render_scfg_info_notebook_center(scfg)

## Early Exit: Python Synthesis

In [27]:
original_ast = unparse_code(while_loop_with_exit)[0]
transformed_ast  = SCFG2ASTTransformer().transform(original=original_ast, scfg=scfg)
display_source(ast.unparse(transformed_ast))

def transformed_while_loop_with_exit(a: int):
    c = 0
    __scfg_loop_cont__ = True
    while __scfg_loop_cont__:
        if c < 10:
            c += 3
            if c > a:
                __scfg_exit_var_0__ = 1
                __scfg_backedge_var_0__ = 1
            else:
                __scfg_backedge_var_0__ = 0
                __scfg_exit_var_0__ = -1
        else:
            __scfg_exit_var_0__ = 0
            __scfg_backedge_var_0__ = 1
        __scfg_loop_cont__ = not __scfg_backedge_var_0__
    if __scfg_exit_var_0__ in (0,):
        __scfg_return_value__ = c
    else:
        __scfg_return_value__ = c + 1
    return __scfg_return_value__

## For-Loop: Example and CFG

The Python for-loop is transformed into a while-loop. In order to achieve this, the syntax must be _desugared_ because the for-loop construct in Python does multiple things:

* Setup the iteration variable
* Initialize the iterator
* Stop the iteration when `StopIteration` is raised
* The iteration variable escapes the loop

In [28]:
    def for_loop():
        c = 0
        for i in range(10):
            c += i
        return c

In [29]:
scfg = AST2SCFGTransformer(for_loop).transform_to_SCFG()

In [30]:
render_scfg_info_notebook_center(scfg)

## For-Loop: Restructure

In [31]:
scfg.restructure()

render_scfg_info_notebook_center(scfg)

## For-loop: Python Synthesis

In [32]:
original_ast = unparse_code(for_loop)[0]
transformed_ast  = SCFG2ASTTransformer().transform(original=original_ast, scfg=scfg)
display_source(ast.unparse(transformed_ast))

def transformed_for_loop():
    c = 0
    __scfg_iterator_1__ = iter(range(10))
    i = None
    __scfg_loop_cont__ = True
    while __scfg_loop_cont__:
        __scfg_iter_last_1__ = i
        i = next(__scfg_iterator_1__, '__scfg_sentinel__')
        if i != '__scfg_sentinel__':
            c += i
            __scfg_backedge_var_0__ = 0
        else:
            __scfg_backedge_var_0__ = 1
        __scfg_loop_cont__ = not __scfg_backedge_var_0__
    i = __scfg_iter_last_1__
    return c

## Break and Continue: Example and CFG

In [33]:
def break_and_continue(x: int, y: int) -> int:
    for i in range(2):
        if i == x:
            i = 3
            return i + 100
        elif i == y:
            i = 4
            break
        else:
            continue
    return i

In [34]:
scfg = AST2SCFGTransformer(break_and_continue).transform_to_SCFG()

In [35]:
render_scfg_info_notebook_center(scfg)

## Break and Continue: restructure

In [36]:
scfg.restructure()

In [37]:
render_scfg_info_notebook_center(scfg)

In [38]:
original_ast = unparse_code(break_and_continue)[0]
transformed_ast  = SCFG2ASTTransformer().transform(original=original_ast, scfg=scfg)
display_source(ast.unparse(transformed_ast))

def transformed_break_and_continue(x: int, y: int) -> int:
    __scfg_iterator_1__ = iter(range(2))
    i = None
    __scfg_loop_cont__ = True
    while __scfg_loop_cont__:
        __scfg_iter_last_1__ = i
        i = next(__scfg_iterator_1__, '__scfg_sentinel__')
        if i != '__scfg_sentinel__':
            if i == x:
                __scfg_exit_var_0__ = 1
                __scfg_backedge_var_0__ = 1
            elif i == y:
                __scfg_exit_var_0__ = 2
                __scfg_backedge_var_0__ = 1
            else:
                __scfg_backedge_var_0__ = 0
                __scfg_exit_var_0__ = -1
        else:
            __scfg_exit_var_0__ = 0
            __scfg_backedge_var_0__ = 1
        __scfg_loop_cont__ = not __scfg_backedge_var_0__
    if __scfg_exit_var_0__ in (0,):
        i = __scfg_iter_last_1__
        __scfg_control_var_0__ = 0
    elif __scfg_exit_var_0__ in (1,):
        i = 3
        __scfg_return_value__ = i + 100
        __scfg_control_var_0__ = 1
    else:
        i = 4
        __scfg_control_var_0__ = 2
    if __scfg_control_var_0__ in (0, 2):
        __scfg_return_value__ = i
    else:
        pass
    return __scfg_return_value__

TODO

* nested loops